In [51]:
# default_exp getdata
# 上面一行用于nbdev中声明本模块的名称。必须是notebook的第一个Cell的第一行。

# getdata

- 描述：抗击新冠病毒（4）-获取并保存在线数据
- 功能：获取数据,转换为列表格式,保存到*.csv文件,用于后续的分析和绘图输出。
- 模块：使用JupyterLab、Python、nbdev等完成。用到的Python模块包括：
  - requests，访问web服务网站。
  - re，正则表达式解析。
  - json，JSON格式解析。
  - BeautifulSoup，HTML格式解析。
  - pprint，格式化输出。
  - pandas，数据表格分析。
- [源码-https://github.com/openthings/anti2020ncov](https://github.com/openthings/anti2020ncov)
- 参考：
  - [JupyterLab-数据实验室](https://my.oschina.net/u/2306127?q=jupyterlab)
  - [文学式编程-nbdev入门教程](https://my.oschina.net/u/2306127/blog/3163740)
  - [抗击新冠病毒（1）-开源软件与数据项目](https://my.oschina.net/u/2306127/blog/3162141)
  - [抗击新冠病毒（2）-基于Jupyter+nbdev的数据分析](https://my.oschina.net/u/2306127/blog/3164083)
  - [抗击新冠病毒（3）-探索在线数据资源](https://my.oschina.net/u/2306127/blog/3164744)

In [ ]:
#hide
from nbdev.showdoc import *

In [28]:
#export 
from bs4 import BeautifulSoup
from parser import * #regex_parser
import re
import json
import time
import logging
import datetime
import requests
import pprint 

#### 预设网络地址和创建请求头。
* 丁香园-新型冠状病毒肺炎疫情实时动态，https://ncov.dxy.cn/ncovh5/view/pneumonia?from=singlemessage&isappinstalled=0
* 中国疾控中心，http://www.chinacdc.cn, 分布图 http://2019ncov.chinacdc.cn/2019-nCoV/
* 全国卫健委-新型冠状病毒肺炎疫情防控，http://www.nhc.gov.cn/xcs/xxgzbd/gzbd_index.shtml
* 全国卫健委-官方网站，http://www.nhc.gov.cn
* 世界卫生组织（WHO），https://www.who.int/zh

In [49]:
# export 
# 使用丁香园的数据。
url = "https://ncov.dxy.cn/ncovh5/view/pneumonia?from=singlemessage&isappinstalled=0"
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'
}

#保存的文件名。
dateof = '20200207'

#### 从Web服务获取网页，并解析为JSON格式。
- [Python之json使用](https://www.cnblogs.com/yanwuliu/p/9593826.html)

In [30]:
#export 

def getweb():
    session = requests.session()
    session.headers.update(headers)
    r = session.get(url)
    soup = BeautifulSoup(r.content, 'lxml')

    #获取省市数据。
    area_information = re.search(r'\[(.*)\]', str(soup.find('script', attrs={'id': 'getAreaStat'})))
    area = json.loads(area_information.group(0))
    return area

#### 保存分省的数据到data/prov_日期.csv文件。

In [31]:
# export 
# 写入文件，分省数据。
def saveprovice(area):
    fprovince = "data/" + "prov_" + dateof + ".csv"
    fp = open(fprovince, "w")
    fp.write("省份,确诊,疑似,治愈,死亡\r")
    for a in area:
        fp.write(a['provinceName']+','+ \
                 str(a['confirmedCount'])+','+ \
                 str(a['suspectedCount'])+','+ \
                 str(a['curedCount'])+','+ \
                 str(a['deadCount'])+ '\r')
    fp.close()
    print("writed to "+ fprovince + "\r\n")

#### 保存分市的数据到文件 data/city_日期.csv。

In [50]:
# export
# 写入文件，分市数据。
def savecity(area):
    fcity = "data/" + "city_" + dateof + ".csv"
    fc = open(fcity, "w")
    fc.write("省份,城市,确诊,疑似,治愈,死亡\r")
    for p in area:
        cities = p['cities']
        for c in cities:
            fc.write(p['provinceName']+','+ \
                     c['cityName']+','+ \
                     str(c['confirmedCount'])+','+ \
                     str(c['suspectedCount'])+','+ \
                     str(c['curedCount'])+','+ \
                     str(c['deadCount'])+'\r')
    fc.close()
    print("writed to "+ fcity + "\r\n")

#### 查看保存的文件。
这里用到Notebook的魔法操作符，参考：
- [IPython 6/Jupyter的magic操作符](https://my.oschina.net/u/2306127/blog/3019986)
- [IPython的Magics魔法操作符](https://my.oschina.net/u/2306127/blog/668079)

In [48]:
!ls -l data

总用量 176
-rw-r--r-- 1 supermap supermap   1445 2月   9 22:49 china.csv
-rw-r--r-- 1 supermap supermap  11840 2月   7 23:02 city_20200207.csv
-rw-r--r-- 1 supermap supermap  12156 2月   8 18:39 city_20200208.csv
-rw-r--r-- 1 supermap supermap  12169 2月  10 18:20 city_20200209.csv
-rw-r--r-- 1 supermap supermap 126285 2月   9 15:09 IMG_7082.JPG
-rw-r--r-- 1 supermap supermap    780 2月   7 23:02 prov_20200207.csv
-rw-r--r-- 1 supermap supermap    784 2月   8 18:39 prov_20200208.csv
-rw-r--r-- 1 supermap supermap    790 2月  10 18:20 prov_20200209.csv


## nbdev 适用工具

In [46]:
# 将notebook转化为python的*.py代码，保存到项目名称的子目录中。

from nbdev.export import *
notebook2script()

Converted 00_digdata.ipynb.
Converted 01_getdata.ipynb.
Converted 10_charts.ipynb.
Converted 10_china.ipynb.
Converted index.ipynb.
